In [19]:
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import read_csv
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import predict
import optuna

dataset=pd.read_csv('SolarPrediction_aligned_Sun.csv',engine='python',nrows=576*15)
dataset.head()

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,8,9/1/2016 12:00:00 AM,00:00:08,2.58,51,30.43,103,77.27,11.25,22020,67080
1,310,9/1/2016 12:00:00 AM,00:05:10,2.83,51,30.43,103,153.44,9.00,22020,67080
2,1206,9/1/2016 12:00:00 AM,00:20:06,2.16,51,30.43,103,142.04,7.87,22020,67080
3,1505,9/1/2016 12:00:00 AM,00:25:05,2.21,51,30.43,103,144.12,18.00,22020,67080
4,1809,9/1/2016 12:00:00 AM,00:30:09,2.25,51,30.43,103,67.42,11.25,22020,67080


In [20]:
#factor=dataset.drop("Radiation",axis=1)
dataset=dataset.drop("Data",axis=1)
dataset=dataset.drop("Time",axis=1)
# factor=factor.drop("TimeSunRise",axis=1)
# factor=factor.drop("TimeSunSet",axis=1)
target=dataset["Radiation"]
print(dataset)
#print(target.head(5))

      UNIXTime  Radiation  Temperature  Pressure  Humidity  \
0            8       2.58           51     30.43       103   
1          310       2.83           51     30.43       103   
2         1206       2.16           51     30.43       103   
3         1505       2.21           51     30.43       103   
4         1809       2.25           51     30.43       103   
...        ...        ...          ...       ...       ...   
8635     27622     160.80           53     30.45       100   
8636     27925     221.67           53     30.45       100   
8637     28223     344.90           54     30.46       100   
8638     28522     365.66           54     30.46       101   
8639     28820     355.57           55     30.46       101   

      WindDirection(Degrees)  Speed  TimeSunRise  TimeSunSet  
0                      77.27  11.25        22020       67080  
1                     153.44   9.00        22020       67080  
2                     142.04   7.87        22020       67080  
3  

In [21]:
dataset=dataset.values
target=target.values
dataset[:,1]=dataset[:,8]
dataset[:,8]=target[:]

In [22]:
def create_dataset(dataset,look_back):
    dataX,dataY=[],[]
    for i in range(len(dataset)-look_back-1):
        #a=dataset第0行i~i+look_back個
        a=dataset[i:(i+look_back),0:9]
        dataX.append(a)
        dataY.append(dataset[i+look_back,8])
    return np.array(dataX),np.array(dataY)

In [23]:
train_size=int(len(dataset)*0.67)
test_size=len(dataset)- train_size
train,test=dataset[0:train_size,:],dataset[train_size:len(dataset),:]
trainX,trainY=create_dataset(train,5)
testX,testY=create_dataset(test,5)

In [24]:
trainX=trainX.reshape(5782,45)
testX=testX.reshape(2846,45)
trainY=trainY.reshape(len(trainY),1)
testY=testY.reshape(len(testY),1)
print(trainX.shape)
print(trainX[0,:])

(5782, 45)
[8.0000e+00 6.7080e+04 5.1000e+01 3.0430e+01 1.0300e+02 7.7270e+01
 1.1250e+01 2.2020e+04 2.5800e+00 3.1000e+02 6.7080e+04 5.1000e+01
 3.0430e+01 1.0300e+02 1.5344e+02 9.0000e+00 2.2020e+04 2.8300e+00
 1.2060e+03 6.7080e+04 5.1000e+01 3.0430e+01 1.0300e+02 1.4204e+02
 7.8700e+00 2.2020e+04 2.1600e+00 1.5050e+03 6.7080e+04 5.1000e+01
 3.0430e+01 1.0300e+02 1.4412e+02 1.8000e+01 2.2020e+04 2.2100e+00
 1.8090e+03 6.7080e+04 5.1000e+01 3.0430e+01 1.0300e+02 6.7420e+01
 1.1250e+01 2.2020e+04 2.2500e+00]


In [25]:
dataset=dataset.astype('float32')
target=target.astype('float32')

scalar1=MinMaxScaler(feature_range=(0,1))
scalar2=MinMaxScaler(feature_range=(0,1))
scalar3=MinMaxScaler(feature_range=(0,1))
scalar4=MinMaxScaler(feature_range=(0,1))
#scalar_dim=dataset[:,1]

#target=dataset[:,1]
trainX=scalar1.fit_transform(trainX)
testX=scalar2.fit_transform(testX)
trainY=scalar3.fit_transform(trainY)
testY=scalar4.fit_transform(testY)

In [26]:
def objective(trial):
    # 定義 SVR 的超參數範圍
    params = {
        'C': trial.suggest_float('C', 0.1, 10),
        'epsilon': trial.suggest_float('epsilon', 0.01, 0.1),
        'kernel': trial.suggest_categorical('kernel', ['linear']),
        'gamma': trial.suggest_categorical('gamma', ['scale', 'auto'])  # 'scale' 根據特徵數據的比例自動選擇，'auto' 根據特徵數選擇
    }

    # 創建 SVR 模型
    model = svm.SVR(**params)

    # 使用交叉驗證評估模型性能
    model.fit(trainX, trainY)

    # 進行預測
    y_pred = model.predict(testX)

    # 計算均方誤差
    mse = mean_squared_error(testY, y_pred)
    return mse

In [27]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# 取得最佳超參數
best_params = study.best_params
print("Best Hyperparameters:", best_params)

[I 2024-01-30 18:52:59,389] A new study created in memory with name: no-name-4160b913-5bcd-4ee1-b99c-1dd94710359d
C:\Users\wang8\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-01-30 18:53:00,105] Trial 0 finished with value: 0.006444602928494212 and parameters: {'C': 4.9266609758426, 'epsilon': 0.09801659273295489, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 0 with value: 0.006444602928494212.
C:\Users\wang8\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-01-30 18:53:01,256] Trial 1 finished with value: 0.005725702127349547 and parameters: {'C': 6.473046972832936

Best Hyperparameters: {'C': 0.10127678320148709, 'epsilon': 0.029240378785064282, 'kernel': 'linear', 'gamma': 'auto'}
